In [1]:
import math
import hashlib
import string
import random
import operator, time, itertools, copy
import numpy as np

# Declare Hyperparameters

In [2]:
# I HAVE TAKEN THIS CODE FROM THE COURSEWORK 1 PASSWORD_FITNESS.PY FILE
# Set population size as 1000
pop_size = 1000
# pop_size = 200
# Set the maximum number of generations to be generated to 1000
n_gen = 1000
# n_gen = 400
# Set Mutation percentage
pMu = 0.2
# Calculate the half of the population
half = int(pop_size/2)
# All permissible characters in the password
option = string.digits + string.ascii_uppercase  + "_"

In [3]:
# I HAVE TAKEN THIS CODE FROM THE COURSEWORK 1 PASSWORD_FITNESS.PY FILE
def get_password(student_username, l=10):
    # Possible characters include upper-case English letters, numbers between 0 and 9 (inclusive),
    # and the underscore symbol
    options = string.digits + string.ascii_uppercase  + "_"

    h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
      s += options[n%len(options)]

    return s[0:l]

# TO DO: replace *** with your EECS username and uncomment the code
student_password = get_password('ec23721')
print(student_password)

_5OW0EHQYX


In [4]:
# I HAVE TAKEN THIS CODE FROM THE COURSEWORK 1 PASSWORD_FITNESS.PY FILE
# Distance function
# We are calculating how good our generated password matches with the actual password
def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        # Square of the absolute difference between two Unicode codes
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score

In [5]:
# I HAVE TAKEN THIS CODE FROM THE COURSEWORK 1 PASSWORD_FITNESS.PY FILE
# Generte the population of the size pop_size
def gen_pop(pop_size):
    population = []
    s = ""

    # Randomly generate each password and append it to the population list
    for i in range(pop_size):
        s = ''.join([random.choice(option) for n in range(10)])
        population.append(s)
    return population

In [6]:
# I HAVE TAKEN THIS CODE FROM THE COURSEWORK 1 PASSWORD_FITNESS.PY FILE
# Upper bound of the distance value
MAX_VALUE = distance_function('0000000000', '__________')

In [7]:
# I HAVE TAKEN THIS CODE FROM THE COURSEWORK 1 PASSWORD_FITNESS.PY FILE
#  Returning the fitness value of the generated passsword in a list of tuples (password, fitness)
def get_normalised_fitness(list_of_phrases, student_password):
    ordered_dict = dict()
    phrase_to_find = student_password
    for ind, phrase in enumerate(list_of_phrases):
        # Return 1 when a candidate matches the true password (string distance between them is zero)
        # if phrase in ordered_dict.keys():
        ordered_dict[phrase] = 1 - distance_function(phrase, phrase_to_find) / MAX_VALUE

    return ordered_dict

In [8]:
'''
The crossover function
Here, we are accepting two parents p1 & p2
We are selecting a random crossover point from the length of parent 1 (0-9) and doing single point crossover
'''
def crossover(p1, p2):
    child1 = ''
    child2 = ''

    '''
    I am choosing a random crossover point within the length of the parent1 size (10) and performing the single point crossover
    '''
    crossover_point = random.randrange(len(p1))

    for i in range(len(p1)):
        '''
        Extracting the genetic information of p1 starting from the crossover_point index until the end of the genome + information
        of p2 starting from the beginning of the genome until the crossover_point index for Child1 and exactly the opposite for child2
        '''
        child1 = p1[crossover_point:] + p2[:crossover_point]
        child2 = p1[:crossover_point] + p2[crossover_point:]

    return child1, child2

In [9]:
'''
The mutation function
I am randomly generating a number between 0 & 1. Then, I am checking if the randomly generated number between 0 & 1 is
less than the mutation rate that we are passing as a parameter to the mutation function.

If the random number is less than the mutation rate, we are slicing the offspring 1 (o1) from the start of the genome
till the index of the mutation rate, adding a random character from the admissible options after the mutation rate
index and appending the rest of the offspring string after the mutation.

The same logic is applied to mutate the offspring 2 as well.

If the randomly generated number is greater than the mutation rate, we are returning the original o1 & o2 without mutating
'''
def mutation(o1, o2, mr):

    mut_rate = int(mr*10)

    if random.uniform(0, 1) < mr:
        o1 = o1[:mut_rate] + random.choice(option) + o1[mut_rate+1:]
        o2 = o2[:mut_rate] + random.choice(option) + o2[mut_rate+1:]
    return o1, o2

In [10]:
'''
The Generate offspring function
'''
def gen_offsprings(pool, pMu):
    # Initialize a population list in which we'll append the newly generated offsprings
    population = []

    # Run the loop number of times equivalent to the population size
    for i in range(pop_size):
        '''
        Get the length of the mating pool which is passed as a parameter for this function.
        Select a random number from the range equivalent to the length of the mating pool using random.randrange
        Using this randomly generated number, get the password from the pool
        Pool is in the format [('10 digit password', fitness), ('10 digit password', fitness), ('10 digit password', fitness)]
        So, parent1 will have pool[randomly generated number][first element of the tuple]
        '''
        p1 = random.randrange(len(pool))
        parent1 = pool[p1][0]
        # parent2 generted same as parent1
        p2 = random.randrange(len(pool))
        parent2 = pool[p2][0]
        # Call the crossover function which will return 2 offsprings. Collect them in offspring1, offspring2
        offspring1, offspring2 = crossover(parent1, parent2)
        # Call the mutation function which will return either the mutated or non-mutated offsprings based on the mutation rate
        mutated_offspring_1, mutated_offspring_2 = mutation(offspring1, offspring2, pMu)
        # Add both these offsprings to the population list
        population.append(mutated_offspring_1)
        population.append(mutated_offspring_2)

    '''
    For each iteration of the above loop, we are generating 2 offsprings.
    So, since my population size is 1000, we are generating twice the offsprings, i.e. 2000
    But, we only need 1000 offsprings.
    So, I'm applying the fitness fucntion to the newly generated population.
    '''
    fitness = get_normalised_fitness(population, student_password)
    '''
    Sort the passwords based on their fitness such that the generated password with the lowest fitness is
    at the start of the list and the password with the highest fitness is at the end.
    '''
    fitness = sorted(fitness.items(), key=lambda cost: cost[1])
    '''
    The selection process:
    As the list is sorted with the highest fitness at the end, we are taking the last pop_size (1000) passwords
    with the highest fitness and discarding the rest with lower fitnesses
    '''
    fitness = fitness[-pop_size:]

    # Return the best 1000 passwords
    return fitness

In [11]:
def gen_algo():
  # A list to get all the generated passwords in
  generated_population = []

  '''
  This code will run unitl it gets the correct password or the length of pop_size.
  '''
  for i in range(n_gen):
      '''
      Check if we are running the loop for the first time
      If this is the first iteration, we need to generate the population.
      '''
      if i == 0:
          # Generate population stored in generated_population
          generated_population = gen_pop(pop_size)

      '''
      Calculate the fitness for the generated population
      The values in generated_population are in the format ['pw1', 'pw2', ..., 'pw(pop_size-1)']
      '''
      fitness = get_normalised_fitness(generated_population, student_password)

      '''
      Sort the passwords based on their fitness such that the generated password with the lowest fitness is
      at the start of the list and the password with the highest fitness is at the end.
      '''
      fitness = sorted(fitness.items(), key=lambda cost: cost[1])

      # Print the password with the best fitness for this generation
      print('Generation {} best match: {} with fitness: {}'.format(i+1, fitness[-1][0], fitness[-1][1]))

      # Check if the fitness of the best password == 1. If it is, we have found the matching password and we break the loop
      if fitness[-1][1] == 1:
          print('\n\nPassword match found!!')
          print('Password generated:', fitness[-1], 'and actual password:', student_password)
          print('====================================================================================\n\n')
          
          return fitness[-1], i+1

      # Create a mating pool that chooses the last best (pop_size/2) elements from the population based on the sorted fitness.
      mating_pool = copy.deepcopy(fitness)[half:]

      # Call the gen_offsprings function which will return the next population by generating the parents, doing crossover and mutation.
      offspring_population = gen_offsprings(mating_pool, pMu)
      '''
      We are implementing the (μ,λ) Evolutionary strategy where we are replacing the parent population with the offspring
      population by using the list.clear() functionality
      '''
      generated_population.clear()

      # Iterate through the offspring_population received and append only the newly generated passwords to the empty generated_population list
      for tpl in range(len(offspring_population)):
          generated_population.append(offspring_population[tpl][0])

  return None

gen_algo()

Generation 1 best match: W9MR0EAV7Y with fitness: 0.7054769963637376
Generation 2 best match: Z5GY1KMRXY with fitness: 0.7788064412694455
Generation 3 best match: WDPG0HGQQX with fitness: 0.748209479481961
Generation 4 best match: _5JW6JMJYW with fitness: 0.8132425605922714
Generation 5 best match: _7QW2EJV_Y with fitness: 0.8345539174463101
Generation 6 best match: _5JW6JGQQX with fitness: 0.8431943636439898
Generation 7 best match: Y1MW0EHQRY with fitness: 0.861290348325533
Generation 8 best match: _5SX0FHQOY with fitness: 0.8809409438496084
Generation 9 best match: _5JW0EJQQX with fitness: 0.9054983215718003
Generation 10 best match: _5QW0GHQYX with fitness: 0.9587431501496483
Generation 11 best match: _5PS0EHQYX with fitness: 0.9562405025506316
Generation 12 best match: _5OW0EHQYX with fitness: 1.0


Password match found!!
Password generated: ('_5OW0EHQYX', 1.0) and actual password: _5OW0EHQYX




(('_5OW0EHQYX', 1.0), 12)

### 3.3 Number of reproductions
#### Report the number of reproductions you had to perform to converge to the true password. Run your code 5-10 times and report the average and standard deviation of the number of reproductions (since Genetic Algorithms involve randomisation).

In [12]:
no_of_reproductions = []
for i in range (10):
  print('Iteration:', i+1, '\n')
  pw_match, gen = gen_algo()
  no_of_reproductions.append(gen)

print('\nStandard Deviation of all the reproductions by running the algorithm 10 times:', np.std(no_of_reproductions))
print('Mean of all the reproductions by running the algorithm 10 times:', np.mean(no_of_reproductions))

Iteration: 1 

Generation 1 best match: Q2OR8LFMXO with fitness: 0.6995448375006884
Generation 2 best match: _0WK4EAPOZ with fitness: 0.7264908992075986
Generation 3 best match: O4OT1NKJWX with fitness: 0.75897171821333
Generation 4 best match: _4XT0EIPZK with fitness: 0.8200375777054005
Generation 5 best match: _4NT0EIPWS with fitness: 0.8631446171909987
Generation 6 best match: _6OT0EIPZK with fitness: 0.8637970751547689
Generation 7 best match: _4NT0EEIYW with fitness: 0.8644545374362888
Generation 8 best match: _5NW6EHPYV with fitness: 0.9144690967247204
Generation 9 best match: _5OX1EHVYX with fitness: 0.938210598047749
Generation 10 best match: _5OX1EHVYX with fitness: 0.938210598047749
Generation 11 best match: _5OT0EIPYX with fitness: 0.9455624440684255
Generation 12 best match: _5RW0FHQYX with fitness: 0.960148943218215
Generation 13 best match: _5NW0EHQXX with fitness: 0.9708270017004211
Generation 14 best match: _5RW0EHQYX with fitness: 0.9747354423680045
Generation 15 best 